## Priporočilni sistem - podobni uporabniki (Collaborative filtering)



In [1]:
import numpy as np
import pandas as pd
import sklearn.cross_decomposition

### 1. Beri podatke o ocenah iz datoteke

Opis podatkovnega seta je : [link](http://files.grouplens.org/datasets/movielens/ml-100k-README.txt).


In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

Pregled podatkov:

In [3]:
#df.head()

Lahko dodamo naslove filmom:

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
#movie_titles.head()

Then merge the dataframes:

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
print(df.shape)
#df.head()

(100003, 5)


#### Vpr 1.
Ugotovi, koliko je ocen, koliko uporabnikov in koliko filmov.

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

#### Sortiramo podatke po uporabnikih


In [7]:
# Kaj prestejemo tukaj ?
df.groupby('user_id')['user_id'].count()

user_id
0        3
1      272
2       62
3       54
4       24
      ... 
939     49
940    107
941     22
942     79
943    168
Name: user_id, Length: 944, dtype: int64

In [8]:
# Ugotovite delovanje
# Uporabniki z najvec ocenami
series_user_nratings = df.groupby('user_id')['user_id'].count()

#series_user_nratings 
#series_user_nratings.values

# Sortirani podatki po user IDju
df4 = df.sort_values('user_id')

id_vsi = series_user_nratings.values
id100 = series_user_nratings.values[0:100]

indeksi_df = df['user_id'].isin(id100)

# Nova serija podatkov
df5 = df.loc[indeksi_df]

#### Vpr 2.
Kakšne podatke predstavlja df5 in kako smo jih dobili ? 

### Training podatki

Podatke običajno razdelimo na učno in testno množico. 
Uporabljali bomo samo učno množico. 

In [22]:
#from sklearn.model_selection import train_test_split
#train_data, test_data = train_test_split(df, test_size=0.25)

# Podatki so naši originalni podatki
train_data = df;

n_users = train_data.user_id.nunique()
n_items = train_data.item_id.nunique()

#print('Num. of Users: '+ str(n_users))
#print('Num of Movies: '+ str(n_items))


Metoda, ki 0 nadomesti z NaN.

In [11]:
def replaceZero(input_arr):
    out_arr = np.copy(input_arr)
    out_arr[out_arr == 0.0] = np.NaN
    return out_arr

Kreiramo matriko uporabniških ocen filmov. Oglej si obe matriki. 

In [12]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

train_data_2 = replaceZero(train_data_matrix)


#### Vpr 3.
Preizkusi in ugotovi, kakšne podatke generira naslednja koda (velikost, kaj vsebujejo)? 

In [13]:
# Sortiraj po vsoti ocen za uporabnika
sm = train_data_matrix.sum(axis = 1)
sm.shape
sort_ind = np.argsort(-sm)
sm2 = sm[sort_ind]

# 
train_data_3 = train_data_2[sort_ind,:]
# 
train_data_4 = train_data_3[0:100,:]

#### Vpr 4.
Preizkusi z različnimi vrednostmi in ugotovi, kaj izračunamo? 

In [14]:
from scipy.stats import pearsonr

# Kako izracunamo korelacijo med dvema uporabnikoma ?

ur_1 = train_data_4[0,:]
ur_2 = train_data_4[3,:]
mask = ~np.isnan(ur_1) & ~np.isnan(ur_2)  
corr, _ = pearsonr(ur_1[mask],ur_2[mask])


In [15]:
# Metoda za podobnost
def calcPearson(data_matr):
    pears = np.zeros([data_matr.shape[0],data_matr.shape[0]])
    for j in range(0, data_matr.shape[0]):
        for i in range(j, data_matr.shape[0]):
            pears[j,i] = np.nan;  # default value
            mask = ~np.isnan(data_matr[j,:]) & ~np.isnan(data_matr[i,:])
            if sum(mask) >= 2:
                pears[j,i], _ = pearsonr(data_matr[j,mask],data_matr[i,mask])
            pears[i,j] = pears[j,i];
        pears[j,j] = 0.0
    return pears

#def user_simil = calcPearson(data_matr):

#### Vpr 5.
Uporabi metodo za izračun korelacije na naših podatkih. Preglej rezultat, velikost in 
vrednosti v dobljeni matriki.
  

In [16]:
# Izracunaj pears_cor
pears_cor = calcPearson(train_data_4)


### Izračun napovedi ocen za enega uporabnika

Enačba:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

In [17]:
# Postopek 

# Uporabljali bomo to spremenljivko
user_similarity = np.copy(pears_cor);

# Povprecni rating uporabnika
mean_user_rating = np.nanmean(train_data_4, axis=1)
# Povpr rating kot stolpcni vektor
c3 = mean_user_rating[:, np.newaxis]
# Razlika rating - povprecje, vsebuje nan
ratings_diff = (train_data_4 - mean_user_rating[:, np.newaxis]) 
# Relativni ratnig, vsebuje 0 kjer ni ocene
rel_ratings = np.nan_to_num(ratings_diff)

# Podobnost z upor. 0
usr1 = user_similarity[:,0]
# Kaj pomeni ?
c2 = usr1.dot(rel_ratings)
# Rezultat
c4 = c2 / np.array([np.abs(usr1).sum()]).T


#### Vpr 6.
Opiši postopek zgoraj za enega uporabnika. Kaj je rezultat ? Dopolni postopek glede na enačbo.

### Za vse uporabnike
Preizkusi in komentiraj postopek. 

In [18]:
i1 = pears_cor[:,:] < 0.0

user_similarity = np.copy(pears_cor);
user_similarity[i1] = 0.0

# To so zdaj vsote abs vrednosti podobnih userjev za vsakega userja
c5 = np.array([np.abs(user_similarity).sum(axis = 0)]).T

# Kaj je to?
calc1 = user_similarity.dot(rel_ratings) # / np.array([np.abs(user_similarity).sum(axis=1)]).T

# ?
calc2 = calc1 / c5

# ? 
calc3 = calc2 + c3

#### Vpr 7.
Kakšen rezultat dobiš za vse uporabnike ? Kaj bo v primeru, če podobnost med uporabnikom ne obstaja ?



#### Dodatna naloga 1
Izračunaj matriko z napakami predvidenih ocen. Izriši histogram teh napak. 


### Evalvacija pravilnosti napovedi sistema

Najpomembnejše merilo za pravilnost napovedi je srednja kvadr. napaka: *Root Mean Squared Error (RMSE)*.
 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />

Uporabimo funkcijo [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) 
iz `sklearn`. 

Druga merila so tukaj [link](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

In [19]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    
    ground_tr_2 = np.nan_to_num(ground_truth);
    prediction2 = np.nan_to_num(prediction);
    pred_valid = prediction2[:,0] > 0.0;

    # samo veljavne vrstice kjer pred ni nan
    prediction3 = prediction2[pred_valid,:]
    ground3 = ground_tr_2[pred_valid,:];
    
    prediction1 = prediction3[ground3.nonzero()].flatten() 
    ground_truth1 = ground3[ground3.nonzero()].flatten()

    return sqrt(mean_squared_error(prediction1, ground_truth1))

In [20]:
# Preizkusi in izpisi

napaka = rmse(calc3, train_data_4);

print("RMSE napaka napovedi: ", napaka)

RMSE napaka napovedi:  0.9477294081486036


#### Vpr 8.
Spreminjaj število sorodnih uporabnikov, ki jih upoštevaš pri izračunih napovedi ocen.
Kako se pri tem spreminja napaka (opis, podatki ali graf) ? 



In [21]:
# Koda


#### Dodatna naloga 2
Izdelaj metodo za izračun napovedi ocen in jo preizkusi z različnimi podatki. 


In [152]:
# Metoda naj vrne tabelo napovedanih ocen.

def predictRating (rating_data, user_similarity):
    
    pred_rating = 0;
    # Povprecni rating uporabnika
    # Povpr rating kot stolpcni vektor
    # Razlika rating - povprecje, vsebuje nan
    # Relativni ratnig, vsebuje 0 kjer ni ocene
    
    return pred_rating

In [153]:
# predictRating(train_data_4, user_similarity)


#### Dodatna naloga 3
Testiraj delovanje sistema (povpr. napaka, čas izračuna), če kot podatke vzameš delno ali kompletno bazo uporabnikov.   


